In [1]:
import os

In [2]:
%pwd

'c:\\Users\\HP\\Car_Price_Prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\HP\\Car_Price_Prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    random_state: 0
    target_column: str

In [6]:
from carpriceproject.constants import *
from carpriceproject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestRegressor
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            random_state= 0,
            target_column = schema.name
            
        )

        return model_trainer_config

In [8]:
import pandas as pd
import os
from carpriceproject import logger
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
import joblib

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import os
import joblib

class ModelTrainer:
    def __init__(self, config):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Initialize LabelEncoder
        label_encoder = LabelEncoder()

        # Encode categorical variables in both train and test data
        categorical_vars = ['name', 'fuel', 'seller_type', 'transmission', 'owner']
        for var in categorical_vars:
            combined_data = pd.concat([train_data[var], test_data[var]], axis=0)
            train_data[var + '_encoded'] = label_encoder.fit_transform(train_data[var])
            test_data[var + '_encoded'] = label_encoder.transform(test_data[var])

        # Define features (X) and target (y)
        features = ['name_encoded', 'year', 'km_driven', 'fuel_encoded', 'seller_type_encoded', 
                    'transmission_encoded', 'owner_encoded', 'mileage', 'engine', 'max_power', 'seats']
        target = 'selling_price'

        train_x = train_data[features]
        train_y = train_data[target]
        test_x = test_data[features]
        test_y = test_data[target]

        # Initialize and train RandomForestRegressor
        model = RandomForestRegressor(random_state=self.config.random_state)
        model.fit(train_x, train_y)

        # Save the trained model
        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))
        print(f"Model trained and saved to {os.path.join(self.config.root_dir, self.config.model_name)}")


In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-07-05 11:19:24,537: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-05 11:19:24,545: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-05 11:19:24,550: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-05 11:19:24,551: INFO: common: created directory at: artifacts]
[2024-07-05 11:19:24,552: INFO: common: created directory at: artifacts/model_trainer]


ValueError: could not convert string to float: 'Petrol'